<a href="https://colab.research.google.com/github/leticiamelom/angular-cli/blob/master/flappy_bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [227]:
pip install pygame

In [228]:
pip install neat-python

In [229]:
import pygame

In [230]:
import neat

In [231]:
import time

In [232]:
import os

In [233]:
import random

In [234]:
TELA_ALTURA = 600
TELA_LARGURA = 800

In [235]:
#imagens dos passaros, scale duplica o tamanho do passaro
bird_imgs = [
        pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird1.png'))),
        pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird2.png'))),
        pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bird3.png')))
]

In [236]:
pipe_img = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'pipe.png')))

In [237]:
base_img = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'base.png')))

In [238]:
bg_img = pygame.transform.scale2x(pygame.image.load(os.path.join('imgs', 'bg.png')))

In [239]:
pygame.font.init()

In [240]:
fonte_pontos = pygame.font.SysFont('arial', 50)

In [241]:
class Bird:
  IMGS = bird_imgs
  ROTACAO_MAX = 25
  VELOC_ROTACAO = 20
  TEMPO_ANIMACAO = 5


In [242]:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.angulo = 0
    self.velocidade = 0
    self.altura = self.y
    self.tempo = 0
    self.contagem_imagem = 0
    self.imagem = IMGS[0]

In [243]:
    def pular (self):
      self.velocidade = -10.5
      self.tempo = 0
      self.altura = self.y

In [244]:
    def mover (self):

      self.tempo += 1
      deslocamento = 1.5 * (self.tempo**2) + self.velocidade * self.tempo

      if deslocamento > 16:
        deslocamento = 16
      elif deslocamento < 0:
         deslocamento -= 2

      self.y += deslocamento

      if  deslocamento < 0 or self.y < (self.altura + 50):
        if self.angulo < self.ROTACAO_MAX:
          self.angulo = self.ROTACAO_MAX
        else:
          if self.angulo > -90:
            self.angulo -= self.VELOC_ROTACAO

In [245]:
    def desenhar(self, tela):

      self.contagem_imagem += 1

      if self.contagem_imagem < self.TEMPO_ANIMACAO:
        self.imagem = self.IMGS[0]
      elif self.contagem_imagem < self.TEMPO_ANIMACAO*2:
        self.imagem = self.IMGS[1]
      elif self.contagem_imagem < self.TEMPO_ANIMACAO*3:
        self.imagem = self.IMGS[2]
      elif self.contagem_imagem < self.TEMPO_ANIMACAO*4:
        self.imagem = self.IMGS[1]
      elif self.contagem_imagem < self.TEMPO_ANIMACAO*4 + 1:
        self.imagem = self.IMGS[0]
        self.contagem_imagem = 0

      if self.angulo <= -80:
        self.imagem = self.IMGS[1]
        self.contagem_imagem = self.TEMPO_ANIMACAO*2

      imagem_rotacionada = pygame.transform.rotate(self.imagem, self.angulo)
      pos_centro_imagem = self.imagem.get_rect(topleft=(self.x, self.y)).center
      retangulo = imagem_rotacionada.get_rect(center=pos_centro_imagem)
      tela.blit(imagem_rotacionada, retangulo.topleft)

In [246]:
    def get_mask(self):
      return pygame.mask.from_surface(self.imagem)

In [247]:
class Pipe:
  DISTANCIA = 200
  VELOCIDADE = 5

In [248]:
  def __init__(self, x):
    self.x = x
    self.altura = 0
    self.pos_topo = 0
    self.pos_base = 0
    self.PIPE_TOPO = pygame.transform.flip(pipe_img, False, True)
    self.PIPE_BASE = pipe_img
    self.passou = False
    self.definir_altura()

In [249]:
    def definir_altura(self):
      self.altura = random.randrange(50, 450)
      self.pos_topo = self.altura - self.PIPE_TOPO.get_height()
      self.pos_base = self.altura + self.DISTANCIA

In [250]:
    def mover(self):
      self.x -= self.VELOCIDADE

In [251]:
    def desenhar(self, tela):
      tela.blit(self.PIPE_TOPO, (self.x, self.pos_topo))
      tela.blit(self.BASE, (self.x, self.base))

In [252]:
    def colidir(self, bird):
      bird_mask = bird.get_mask()
      topo_mask = pygame.mask.from_surface(self.PIPE_TOPO)
      base_mask = pygame.mask.from_surface(self.PIPE_BASE)

      distancia_topo = (self.x - bird.x, self.pos_topo - round(bird.y))
      distancia_base = (self.x - bird.x, self.pos_base - round(bird.y))

      topo_ponto = bird_mask.overlap(topo_mask, distancia_topo)
      base_ponto = bird_mask.overlap(base_mask, distancia_base)

      if base_ponto or topo_ponto:
        return True
      else:
        return False

In [253]:
class Chao:
  VELOCIDADE = 5
  LARGURA = base_img.get_width()
  IMAGEM = base_img

In [254]:
  def __init__(self, y):
    self.y = y
    self.x1 = 0
    self.x2 = self.LARGURA

In [255]:
  def mover(self):
    self.x1 -= self.VELOCIDADE
    self.x2 -= self.VELOCIDADE

    if self.x1 + self.LARGURA < 0:
      self.x1 = self.x2 + self.LARGURA
    if self.x2 + self.LARGURA < 0:
      self.x2 = self.x1 + self.LARGURA

In [256]:
   def desenhar(self, tela):
     tela.blit(self.IMAGEM, (self.x1, self.y))
     tela.blit(self.IMAGEM, (self.x2, self.y))

In [257]:
def desenhar_tela(tela, birds, pipes, base, pontos):
  tela.blit(bg_img, (0, 0))
  for bird in birds:
    bird.desenhar(tela)
  for pipe in pipes:
    pipe.desenhar(tela)

  texto = FONTE_PONTOS.render(f"Pontuação: {pontos}", 1, (255, 255, 255))
  tela.blit(texto, (TELA_LARGURA - 10 - texto.get.width(), 10))
  base.desenhar(tela)
  pygame.display.update()

In [258]:
def main():
  birds = [Bird(230, 350)]
  base = Base(730)
  pipes = [Pipe(700)]
  tela = pygame.display.set_mode((TELA_LARGURA, TELA_ALTURA))
  pontos = 0
  relogio = pygame.time.Clock()

  rodando = True
  while rodando:
    relogio.tick(30)

    for evento in pygame.event.get():
      if evento.type == pygame.QUIT:
        rodando = False
        pygame.quit()
        quit
      if evento.type == pygame.KEYDOWN:
        if evento.key == pygame.K_SPACE:
          for bird in birds:
            bird.pular()

    for bird in birds:
      bird.mover()
    base.mover()

    adicionar_pipe = False
    remover_pipes = []
    for pipe in pipes:
      for i, bird in enumerate(birds):
        if pipe.colidir(bird):
          birds.pop(i)
        if not pipe.passou and bird.x > pipe.x:
          pipe.passou = True
          adicionar_pipe = True
      pipe.mover()
      if pipe.x + pipe.PIPE_TOPO.get_width() < 0:
        remover_pipes.append(pipe)

      if adicionar_pipe:
        pontos += 1
        pipes.append(Pipe(600))
      for pipe in remover_pipes:
        pipes.remove(pipe)

      for i, bird in enumerate(birds):
        if (bird.y + bird.imagem.get_height()) > base.y or bird.y < 0:
          birds.pop(i)

      desenhar_tela(tela, birds, pipes, base, pontos)

In [259]:
if __name__ == '__run__':
  run()